# About



`ITS_LIVE_TOOL` is a package designed to aid users working with the [Inter-mission Time Series of Land Ice Velocity and Elevation](link) (ITS_LIVE) dataset. The package provides functions for accessing data as well as various methods to process ITS_LIVE observations. This notebook will demonstrate various elements of the package and walk through the steps of a typical workflow using ITS_LIVE_TOOL. 

## Overview 

```{note}
keep this overview here? or move to readme doc?
```
The tools in the package fall into two main categories: 1) data access + organization, 2) data processing. 

### Data Access + Organization

#### 1. Interactive map widget

This is an interactive widget design to streamline access ITS_LIVE image pair ice velocity time series. 

#### 2. `Glacier`, `Glacier_Centerline`, `Glacier_Point` objects

These are provided to store and keep track of different types of data related to individual units of analysis such as points, centerlines or full glacier surface areas. 

This roadmap document will first demonstrate data access and organization tools before demonstrating the processing tools. 

### Data Processing

We demonstrate and make available two processing routines. Be sure to check out the accompanying [book]() and consider if either of these are appropriate for your data and use case. Note that these methods are in active development and thus should be considered in *beta* phase. Please perform your own data inspection and due diligence if implementing these methods. 

#### Inversion

**to add**
Description -- link to full description and examples


#### Gaussian Process Regression 

**to add**
Description -- link to full description and examples


### Processing

We demonstrate and make available two processing routines. Be sure to check out the accompanying [book]() and consider if either of these are appropriate for your data and use case. Note that these methods are in active development and thus should be considered in *beta* phase. Please perform your own data inspection and due diligence if implementing these methods. 

In [1]:
from ITS_LIVE_TOOL import datacube_tools, interactive, obj_setup

In [2]:
import os
import numpy as np
import pyproj
import matplotlib.path as path
import s3fs
import zarr
import matplotlib.pyplot as plt
import scipy
from datetime import timedelta
from tqdm import tqdm
import xarray as xr
import re
import pandas as pd
import geopandas as gpd
import matplotlib.path as mplp
import ipyleaflet as ipyl
from ipyleaflet import WMSLayer
import ipywidgets as ipyw
import json
import pandas as pd
from ipyleaflet import Map, WMSLayer, basemaps
from ipywidgets import HTML
from owslib.wms import WebMapService

## Install

Install this package using the below command 

```{note}
someday we hope to have a pip or conda install, for now use pip install git+ github repo url
```

# Section 1: Data Access + Organization

### How to use

There are two ways to access data using ITS_LIVE_TOOL. The first way is through the interactive widget. This is great for exploratory analysis. The second way is by specifying an RGI ID and point coordinates manually. This is useful if you already know which glacier(s) you want to examine and simply want to pass a list of RGI IDs and coordinates. We'll first demonstrate using the interactive widget.

### Interactive data selection widget

First, use the interactive map to select data. Do this by right- and left-clicking on the map location where you'd like to access data. 
This will return an object containing the coordinates of the point you clicked, a `geopandas.GeoDataFrame` of the [RGI7](http://www.glims.org/rgi_user_guide/welcome.html) data for that glacier and the URL of the ITS_LIVE granule covering the clicked point. 

```{note}
If the glacier you want to study lies in multiple ITS_LIVE granules, or you'd like to look at multiple glaciers, click in multiple points and the output objects will be appended with each click. 
```

In [3]:
data_map = interactive.Widget()

In [4]:
data_map.display()

### Create data objects

Once you have made your selection(s) on the map, use the following methods to create objects that will store your data

#### If you've only selected one glacier, create individual objects as follows: 

In [5]:
data_map.urls

['http://its-live-data.s3.amazonaws.com/datacubes/v2/N50W140/ITS_LIVE_vel_EPSG3413_G0120_X-3350000_Y350000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v2/N60W140/ITS_LIVE_vel_EPSG3413_G0120_X-3250000_Y350000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v2/N50W130/ITS_LIVE_vel_EPSG3413_G0120_X-3350000_Y250000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v2/N60W130/ITS_LIVE_vel_EPSG3413_G0120_X-3250000_Y250000.zarr']

In [6]:
glacier = obj_setup.create_glacier_from_click(data_map, 0)

In [7]:
glacier.itslive_url

'http://its-live-data.s3.amazonaws.com/datacubes/v2/N50W140/ITS_LIVE_vel_EPSG3413_G0120_X-3350000_Y350000.zarr'

In [48]:
glacier_point = obj_setup.create_glacier_point_from_click(data_map, 0, 'phony label')

original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
subset and load at  63.19 seconds



KeyboardInterrupt



In [23]:
glacier_centerline = obj_setup.create_glacier_centerline_from_click(data_map, 0)

As you can see, these objects all store data related to the same glacier that was selected, but at different spatial scales. The `glacier` object contains an RGIID, name and the RGI outline and attributes for that glacier. 

In [27]:
glacier.outline_prj

,id,CENLON,ZMAX,BGNDATE,ZMIN,RGIID,ASPECT,CENLAT,SLOPE,ZMED,...,TERMTYPE,O2REGION,STATUS,ENDDATE,FORM,SURGING,GLIMSID,O1REGION,NAME,geometry
0,RGI_Alaska.13631,-142.072,5381,20100910,3,RGI60-01.13635,273,60.461,9,1525,...,2,5,0,-9999999,0,9,G217928E60461N,1,Bering Glacier,"MULTIPOLYGON (((506808.975 6723108.546, 506874..."


The `glacier_point` object also has the RGI ID and name inherited from the map widget but additionally contains ITS_LIVE image pair velocity time series data for the selected point as well as a 3x3 pixel cube surrounding the point. 

In [29]:
glacier_point.cube_around_point

<xarray.Dataset>
Dimensions:                (mid_date: 21894, x: 3, y: 4)
Coordinates:
  * mid_date               (mid_date) datetime64[ns] 2020-10-27T20:55:56.2010...
  * x                      (x) float64 -3.247e+06 -3.247e+06 -3.247e+06
  * y                      (y) float64 4.593e+05 4.592e+05 4.591e+05 4.589e+05
    step_count             (mid_date) int64 0 5 12 14 ... 76928 76941 76952
    mapping                int64 0
Data variables:
    v                      (mid_date, y, x) float32 nan 182.0 nan ... 11.0 nan
    vy                     (mid_date, y, x) float32 nan -131.0 nan ... 11.0 nan
    vx                     (mid_date, y, x) float32 nan -126.0 nan ... 2.0 nan
    v_error                (mid_date, y, x) float32 nan 53.0 nan ... nan 6.0 nan
    satellite_img1         (mid_date) object '2A' '2A' '2A' '2B' ... '7' '7' '7'
    satellite_img2         (mid_date) object '2A' '2A' '2B' '2B' ... '8' '8' '8'
    acquisition_date_img1  (mid_date) datetime64[ns] 2020-10-17T20:55:01 ... ...
    acquisition_date_img2  (mid_date) datetime64[ns] 2020-11-06T20:56:51.0000...
    img_separation         (mid_date) float64 21.0 500.0 535.0 ... 217.0 312.0
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               25-Sep-2023 22:55:04
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

The `glacier_centerline` object contains the same RGIID and name information as the first two objects. In addition, it contains OGGM centerline data that are stored as geopandas GeodataFrames.

In [31]:
glacier_centerline.main_centerline

,RGIID,SEGMENT_ID,LE_SEGMENT,MAIN,geometry
19617,RGI60-01.13635,40,212562.0,1,"LINESTRING (522733.337 6719455.665, 522730.077..."


Each of these objects are meant to act as cotainers to intuitively and efficiently store different types of data as you process and continue to work with ITS_LIVE data. 

The previous steps demonstrated the data access and organization functionality of ITS_LIVE_TOOl. The subsequent sections of this notebook will demonstrate different processing methodologies for working with this dataset. 

#### If you've selected multiple glaciers, follow these examples:

In [15]:
glacier0, glacier1 = obj_setup.create_multiple_glacier_objs(data_map)[0], obj_setup.create_multiple_glacier_objs(data_map)[1]


In [16]:
glacier_point_ls = obj_setup.create_multiple_glacier_point_objs(data_map)
glacier_pt0, glacier_pt1 = glacier_point_ls[0], glacier_point_ls[1]

original xy [-141.34499544222683, 60.45529919026818] 4326 maps to datacube (-3250454.7365773427, 361437.5235576865) EPSG:3413
original xy [-141.34499544222683, 60.45529919026818] 4326 maps to datacube (-3250454.7365773427, 361437.5235576865) EPSG:3413
subset and load at  58.26 seconds
original xy [-141.32575672759108, 60.65848956101137] 4326 maps to datacube (-3227246.8706271336, 357759.90272181097) EPSG:3413
original xy [-141.32575672759108, 60.65848956101137] 4326 maps to datacube (-3227246.8706271336, 357759.90272181097) EPSG:3413
subset and load at  60.11 seconds


### Manually create `Glacier`, `Glacier_Point`, `Glacier_Centerline` objects

Create objects that contain the clicked data from the above map widget.  
These are `coords`, `gpdf`, `urls`  


In [33]:
coords, gpdf, urls = obj_setup.return_clicked_info(data_map)

You have 2 glaciers selected


In [34]:
coords

[[60.37791914446092, -143.04795214410368],
 [60.36705675717361, -143.64435229781114]]

In [35]:
label = 'label'
var_ls = ['v','vy','vx','v_error','mapping','satellite_img1','satellite_img2','acquisition_date_img1', 'acquisition_date_img2']


In [41]:
glacier_manual = obj_setup.Glacier(gpdf[0]['NAME'].iloc[0], gpdf[0]['RGIID'].iloc[0], str(gpdf[0].estimate_utm_crs()), 'manual', rgi_outline_from_widget = None)


In [42]:
glacier_manual.name

'Bering Glacier'

In [43]:
glacier_manual.rgi_id

'RGI60-01.13635'

In [44]:
glacier_manual.outline_prj.crs

<Projected CRS: EPSG:32607>
Name: WGS 84 / UTM zone 7N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 144°W and 138°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Yukon. United States (USA) - Alaska (AK).
- bounds: (-144.0, 0.0, -138.0, 84.0)
Coordinate Operation:
- name: UTM zone 7N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [45]:
glacier_pt = obj_setup.Glacier_Point(glacier.name, 'acc. zone', glacier.rgi_id, [coords[0][1], coords[0][0]], var_ls)


original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
subset and load at  62.53 seconds


In [46]:
glacier_centerline = obj_setup.Glacier_Centerline('Jorge Montt', 'RGI60-17.06074')

### `Glacier_Point`

This object inherits name and RGIID attributes from the `Glacier` object described aboev (or you can input them manually). It takes the coordinate information specified in the widget and list of variables of interest and returns an object containing ITS_LIVE image pair time series datasets for the specified point anda 3x3 pixel cube surrounding the specified point. Check out the docs for a description of all of the attributes and methods associated with this object. 

In [12]:
coords

[[61.445551290803365, -147.96202722979135]]

In [13]:
glacier_pt = obj_setup.Glacier_Point(glacier.name, 'acc. zone', glacier.rgi_id, [coords[0][1], coords[0][0]], var_ls)


original xy [-147.96202722979135, 61.445551290803365] 4326 maps to datacube (-3075868.4013177734, 707973.3167825146) EPSG:3413
original xy [-147.96202722979135, 61.445551290803365] 4326 maps to datacube (-3075868.4013177734, 707973.3167825146) EPSG:3413
subset and load at  48.89 seconds


In [14]:
glacier_pt.cube_around_point

<xarray.Dataset>
Dimensions:                (mid_date: 13831, x: 3, y: 3)
Coordinates:
  * mid_date               (mid_date) datetime64[ns] 2020-02-26T21:15:44.1906...
  * x                      (x) float64 -3.076e+06 -3.076e+06 -3.076e+06
  * y                      (y) float64 7.081e+05 7.079e+05 7.078e+05
    step_count             (mid_date) int64 1 6 7 8 ... 53266 53268 53273 53285
    mapping                int64 0
Data variables:
    v                      (mid_date, y, x) float32 nan nan nan ... nan nan nan
    vy                     (mid_date, y, x) float32 nan nan nan ... nan nan nan
    vx                     (mid_date, y, x) float32 nan nan nan ... nan nan nan
    v_error                (mid_date, y, x) float32 nan nan nan ... nan nan nan
    satellite_img1         (mid_date) object '2B' '5' '2B' '7' ... '7' '8' '8'
    satellite_img2         (mid_date) object '2B' '5' '2B' '7' ... '8' '8' '8'
    acquisition_date_img1  (mid_date) datetime64[ns] 2019-06-26T21:15:19.0000...
    acquisition_date_img2  (mid_date) datetime64[ns] 2020-10-28T21:16:09 ... ...
    img_separation         (mid_date) float64 491.0 305.0 340.0 ... 352.0 17.0
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               03-Oct-2023 04:28:15
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

In [42]:
glacier_pt

<xarray.Dataset>
Dimensions:                (mid_date: 3906, y: 83, x: 83)
Coordinates:
    mapping                int64 0
  * mid_date               (mid_date) datetime64[ns] 2020-01-26T14:27:11.2001...
  * x                      (x) float64 6.017e+05 6.019e+05 ... 6.116e+05
  * y                      (y) float64 4.633e+06 4.633e+06 ... 4.623e+06
    step_count             (mid_date) int64 7 29 30 38 ... 31281 31283 31302
Data variables:
    v                      (mid_date, y, x) float32 37.0 25.0 9.0 ... nan nan
    vy                     (mid_date, y, x) float32 -2.0 -11.0 2.0 ... nan nan
    vx                     (mid_date, y, x) float32 36.0 23.0 9.0 ... nan nan
    v_error                (mid_date, y, x) float32 26.0 26.0 26.0 ... nan nan
    satellite_img1         (mid_date) object '2A' '2A' '1A' '2A' ... '8' '8' '8'
    satellite_img2         (mid_date) object '2A' '2A' '1A' '2B' ... '7' '9' '8'
    acquisition_date_img1  (mid_date) datetime64[ns] 2020-01-01T14:27:30.9999...
    acquisition_date_img2  (mid_date) datetime64[ns] 2020-02-20T14:26:51 ... ...
    img_separation         (mid_date) float64 50.0 260.0 12.0 ... 536.0 16.0
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               25-Sep-2023 21:36:13
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/S40W070/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/S40W070/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

### `Glacier_Centerline`

Similarly to the `Glacier_Point` object, a `Glacier_Centerline` object inherits some basic information from a `Glacier` object, but this can also be specified manually if you don't want to create a `Glacier` method beforehand. A `Glacier_Centerline` object has attributes called `centerlines` and `main_centerline`. These are both `Geopandas.GeoDataframe` objects containing OGGM flowline data associated with this glacier. They also contain ITS_LIVE image pair velocity time series clipped to the centerline (**still being implemented**).

In [43]:
glacier_centerline = obj_setup.Glacier_Centerline('Jorge Montt', 'RGI60-17.06074')

In [44]:
glacier_centerline

Each of these objects are meant to act as cotainers to intuitively and efficiently store different types of data as you process and continue to work with ITS_LIVE data. 

The previous steps demonstrated the data access and organization functionality of ITS_LIVE_TOOl. The subsequent sections of this notebook will demonstrate different processing methodologies for working with this dataset. 

# Section 2: Data Processing

```{note}
Put inversion and other processing steps here? 
```